# 1. Import the necessary libraries for you to be able to query 4square API

In [23]:
import requests
import json
import pandas as pd
from getpass import getpass
import folium
from pymongo import MongoClient

In [24]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [27]:
lat = 41.3930198
lon = 2.1855589

def requests_for_foursquare(query, lat, lon, radius=500, limit=10):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat},{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except Exception as e:
        print("Error:", e)
        return None


res = requests_for_foursquare("coffee", lat, lon, radius=500, limit=10)
res

{'results': [{'fsq_id': '5cf510f0031320002cbd5af8',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 57,
   'geocodes': {'drop_off': {'latitude': 41.39243, 'longitude': 2.185926},
    'main': {'latitude': 41.392556, 'longitude': 2.185831},
    'roof': {'latitude': 41.392556, 'longitude': 2.185831}},
   'link': '/v3/places/5cf510f0031320002cbd5af8',
   'location': {'address': 'Calle de Buenaventura Muñoz, 31',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Call

In [28]:
names = [(res["results"][i]["name"]) for i in range(10)]
location = [(res["results"][i]["location"]["address"]) for i in range(10)]
new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
df_coffe_shop = pd.DataFrame(new_dict)
df_coffe_shop

,name,location
0,Orval Barcelona,"Calle de Buenaventura Muñoz, 31"
1,Santa Fe,"Lington, 17"
2,Xocoxavi,Wellington 100-104
3,Granja Itaca,"Roger de Flor, 32"
4,Sacher Café,"Carrer de Llull, 20"
5,Flora Cafe,"Pujades, 21"
6,Boheme Artesans del Pa,"Carrer de Roger de Flor, 34"
7,Starbucks,"Avenida de Vilanova, 3"
8,Bracafe,"Calle Ribes, 29"
9,Nàpols Forn Degustació,"Nàpols, 96"


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [30]:
lat = 41.3930198
lon = 2.1855589

def requests_for_foursquare(query, lat, lon, limit=5):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat},{lon}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except Exception as e:
        print("Error:", e)
        return None


coffee_shop = requests_for_foursquare("coffee", lat, lon, limit=5)
coffee_shop

{'results': [{'fsq_id': '5b8521d489e490002cefef18',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'short_name': 'Coffee Shop',
     'plural_name': 'Coffee Shops',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 501,
   'geocodes': {'main': {'latitude': 41.397168, 'longitude': 2.183123},
    'roof': {'latitude': 41.397168, 'longitude': 2.183123}},
   'link': '/v3/places/5b8521d489e490002cefef18',
   'location': {'address': 'Ausiàs Marc, 151',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
    'locality': 'Barcelona',
    'postcode': '08013',
    '

In [31]:
names = [(res["results"][i]["name"]) for i in range(5)]
location = [(res["results"][i]["location"]["address"]) for i in range(5)]
lat = [(res["results"][i]["geocodes"]["main"]["latitude"]) for i in range (5)]
lon = [(res["results"][i]["geocodes"]["main"]["longitude"]) for i in range (5)]
new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
new_dict["lat"] = lat
new_dict["lon"] = lon

coffee_shops = pd.DataFrame(new_dict)
coffee_shops

,name,location,lat,lon
0,Orval Barcelona,"Calle de Buenaventura Muñoz, 31",41.392556,2.185831
1,Santa Fe,"Lington, 17",41.392081,2.185666
2,Xocoxavi,Wellington 100-104,41.392900,2.185177
3,Granja Itaca,"Roger de Flor, 32",41.391189,2.182878
4,Sacher Café,"Carrer de Llull, 20",41.391850,2.189843


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [35]:
def foursquare_places(venue, token):
    endpoint = f'https://api.foursquare.com/v3/places/search?query={venue}'

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    try:
        response = requests.get(endpoint, headers=headers)
        response.raise_for_status()  
        data = response.json()
        venues = data['results']
        results = []
        for venue in venues:
            name = venue['name']
            location = venue['geocodes']['main']
            lat = location.get('latitude')
            lng = location.get('longitude')
            results.append({
                'name': name,
                'lat': lat,
                'lng': lng
            })
        return results
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []
    
def export_to_json(results, output_directory, output_file):
    try:
        full_file_path = os.path.join(output_directory, output_file)
        os.makedirs(output_directory, exist_ok=True)
        with open(full_file_path, 'w') as json_file:
            json.dump(results, json_file, indent=4)
        print(f"Data exported to '{full_file_path}'")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False
    

Fer example:

get_bookstores_madrid("starbucks", ["lat", "lon"])
​
    if venue == "starbucks":
        requests.get(f"url.....starbucks)
                     
    elif venue == "restaurants":
        requests.get(f"url.....restaurants)
          
    new_list = []
    for i in res:
                                   
        dict_ = {"name":name,
                "lat": lat,
                "lon": lon}
                     
        nre_list.append(dict_)
                     
    return new_list

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [36]:
results = foursquare_places("caffe", token)
output_directory = 'data/'
output_file_name = 'coffe_shops.json'

result = export_to_json(results, output_directory, output_file_name)

Data exported to 'data/coffe_shops.json'


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [38]:
client = MongoClient("mongodb://localhost:27017")

db = client["Ironhack"]

data_file = 'data/coffe_shops.json'

collection_name = "ironcoffee"

with open(data_file, 'r') as json_file:
    data = json.load(json_file)

collection = db[collection_name]
collection.insert_many(data)

client.close()